In [1]:
%%capture
!pip install datasets
!pip install transformers

import json
from datasets import load_dataset, concatenate_datasets, DatasetDict, Dataset
import pandas as pd

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
print(tokenizer.pad_token, tokenizer.eos_token)

<|finetune_right_pad_id|> <|end_of_text|>


# Load dataset

In [4]:
dataset_0 = load_dataset("CohereForAI/aya_collection_language_split", "nepali", split="train")
dataset_1 = load_dataset("CohereForAI/aya_collection_language_split", "nepali", split="test")
dataset_2 = load_dataset("CohereForAI/aya_collection_language_split", "nepali", split="validation")

Generating train split:   0%|          | 0/3576367 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/272872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/274905 [00:00<?, ? examples/s]

In [5]:
dataset_0 = dataset_0.to_pandas()
dataset_1 = dataset_1.to_pandas()
dataset_2 = dataset_2.to_pandas()

# Question answering

In [6]:
df_0 = dataset_0.drop(dataset_0[dataset_0['task_type'] != 'question-answering'].index)
df_1 = dataset_1.drop(dataset_1[dataset_1['task_type'] != 'question-answering'].index)
df_2 = dataset_2.drop(dataset_2[dataset_2['task_type'] != 'question-answering'].index)

In [7]:
df_question_answer = pd.concat([df_0, df_1, df_2])

In [8]:
dataset_question = Dataset.from_pandas(df_question_answer)

# Generation

In [ ]:
df_0 = dataset_0.drop(dataset_0[dataset_0['task_type'] != 'generation'].index)
df_1 = dataset_1.drop(dataset_1[dataset_1['task_type'] != 'generation'].index)
df_2 = dataset_2.drop(dataset_2[dataset_2['task_type'] != 'generation'].index)

df_generation = pd.concat([df_0, df_1, df_2])

In [ ]:
len(df_generation)

In [ ]:
dataset_generation = Dataset.from_pandas(df_generation)

# Prompt

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task in Nepali language. Write a response in Nepali language that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["inputs"]
    outputs      = examples["targets"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
dataset_generation = dataset_generation.map(formatting_prompts_func, batched = True,)
dataset_question = dataset_question.map(formatting_prompts_func, batched = True,)

In [14]:
merged_dataset = concatenate_datasets([dataset_generation, dataset_question])
final_datasets = merged_dataset.remove_columns([col for col in merged_dataset.column_names if col not in ["text"]])

In [15]:
final_datasets

Dataset({
    features: ['text'],
    num_rows: 756546
})

In [16]:
final_datasets = final_datasets.shuffle()
final_datasets = final_datasets.shuffle()
final_datasets = final_datasets.shuffle()

In [17]:
final_datasets.push_to_hub("Rajeeb321/main_dataset_finetune-0",token="hf_")

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/379 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/379 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Rajeeb321/main_dataset_finetune-0/commit/aab1c82f3b9195bc87d797533c7e9f447c719d51', commit_message='Upload dataset', commit_description='', oid='aab1c82f3b9195bc87d797533c7e9f447c719d51', pr_url=None, pr_revision=None, pr_num=None)

# Blend dataset

In [3]:
dataset_blend = load_dataset("Open-Orca/OpenOrca", split="train[3%:4%]")
dataset_base = load_dataset("Rajeeb321/main_dataset_finetune-0", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split:   0%|          | 0/756546 [00:00<?, ? examples/s]

In [8]:
len(dataset_blend)

42339

In [4]:
alpaca_prompt = """{}

### Question:
{}

### Answer:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["system_prompt"]
    inputs       = examples["question"]
    outputs      = examples["response"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass
dataset_blend = dataset_blend.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/42339 [00:00<?, ? examples/s]

In [5]:
merged_dataset = concatenate_datasets([dataset_blend, dataset_base])
final_datasets = merged_dataset.remove_columns([col for col in merged_dataset.column_names if col not in ["text"]])

In [6]:
final_datasets

Dataset({
    features: ['text'],
    num_rows: 798885
})

In [7]:
final_datasets = final_datasets.shuffle()
final_datasets = final_datasets.shuffle()
final_datasets = final_datasets.shuffle()

In [9]:
final_datasets.push_to_hub("Rajeeb321/main_dataset_for_finetune-0",token="hf_")

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/400 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/400 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Rajeeb321/main_dataset_for_finetune-0/commit/3e860d005690e1744210d97bb4cfaeb68abb5fc9', commit_message='Upload dataset', commit_description='', oid='3e860d005690e1744210d97bb4cfaeb68abb5fc9', pr_url=None, pr_revision=None, pr_num=None)